# Stock Data Analysis 

### By Clark Lu, June 2020

### The goal of this project to understand more about the stocks' trends.

### This unfinished project has 3 parts.

### (1) Cleaned and normalized the raw data fetched from www.finam.ru.

### (2) Analyzed and pre-processed the stock price data using Fast Fourier Transformation.

### (3) Clustered the price trends using Machine Learning clustering algorithms.

### Part 1

In [ ]:
#Cleaning and normalizing data from finam.ru

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from datetime import date, timedelta
from dateutil import rrule
from datetime import datetime, timedelta
import pandas as pd

def data_modify(year):
    start = date(year, 1, 3)
    end = date(year, 1, 4)
    day = timedelta(days=1)

    outF = open("SPY" + str(year) + "_all.txt", "a")
    outF.write("<DATE>;<TIME>;<OPEN>;<OPEN_normalized>;<HIGH>;<LOW>;<CLOSE>;<VOL>")
    outF.write("\n")
    outF.write("\n")

    data = pd.read_csv('/home/clarklu/Downloads/SPY' + str(year)[2:] + '.txt', sep=",", header=None)
    data.columns = ['TICKER', 'PER', 'DATE', 'TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']

    #print(nineFour_open)

    a = 0

    mydate = start
    while mydate < end:
        curDay = "{date.year}{date.month:02}{date.day:02}".format(date=mydate)
        mydate += day

        nineFour = data.loc[data['DATE'].isin([curDay])]
        #print(nineFour)
        time_series = nineFour['TIME'].tolist()
        #print(time_series)
        
        not_missing_successive = 1
        if len(time_series) < 380:
            for i in range(3, len(time_series) - 1):
                if (time_series[i + 1] - time_series[i] > 1000):
                    not_missing_successive = 0
        
        print(not_missing_successive)
        
        if not_missing_successive:
            nineFour_open = nineFour['OPEN']
            nineFour_high = nineFour['HIGH']
            nineFour_low = nineFour['LOW']
            nineFour_close = nineFour['CLOSE']
            nineFour_vol = nineFour['VOL']


            if len(nineFour_open) > 390:
                nineFour_open_fixed = nineFour_open[:390]
                nineFour_high_fixed = nineFour_high[:390]
                nineFour_low_fixed = nineFour_low[:390]
                nineFour_close_fixed = nineFour_close[:390]
                nineFour_vol_fixed = nineFour_vol[:390]

            elif len(nineFour_open) < 390:
                nineFour_open_fixed = nineFour_open.tolist()
                nineFour_high_fixed = nineFour_high.tolist()
                nineFour_low_fixed = nineFour_low.tolist()
                nineFour_close_fixed = nineFour_close.tolist()
                nineFour_vol_fixed = nineFour_vol.tolist()

                #print(nineFour_open_fixed)
                i = len(nineFour_open_fixed)

                x = list(range(0, len(nineFour_open_fixed)))
                x = np.array(x).reshape((-1, 1))
                y = nineFour_open_fixed
                yy = nineFour_high_fixed
                yyy = nineFour_low_fixed
                yyyy = nineFour_close_fixed
                yyyyy = nineFour_vol_fixed

                y = np.array(y)
                yy = np.array(yy)
                yyy = np.array(yyy)
                yyyy = np.array(yyyy)
                yyyyy = np.array(yyyyy)

                x_ = PolynomialFeatures(degree=6, include_bias=False).fit_transform(x)
                model = LinearRegression().fit(x_, y)
                model_high = LinearRegression().fit(x_, yy)
                model_low = LinearRegression().fit(x_, yyy)
                model_close = LinearRegression().fit(x_, yyyy)
                model_vol = LinearRegression().fit(x_, yyyyy)

                while i < 390:
                    toAdd = model.predict([[1,i**1,i**2,i**3,i**4,i**5]])
                    toAdd_high = model_high.predict([[1,i**1,i**2,i**3,i**4,i**5]])
                    toAdd_low = model_low.predict([[1,i**1,i**2,i**3,i**4,i**5]])
                    toAdd_close = model_close.predict([[1,i**1,i**2,i**3,i**4,i**5]])
                    toAdd_vol = model_vol.predict([[1,i**1,i**2,i**3,i**4,i**5]])

                    #print(toAdd)
                    nineFour_open_fixed.append(toAdd)
                    nineFour_high_fixed.append(toAdd_high)
                    nineFour_low_fixed.append(toAdd_low)
                    nineFour_close_fixed.append(toAdd_close)
                    nineFour_vol_fixed.append(toAdd_vol)

                    i = i + 1

            #print(nineFour_open_fixed)
            check_list = isinstance(nineFour_open_fixed, list)
            if not check_list:
                nineFour_open_fixed = nineFour_open_fixed.tolist()
                nineFour_high_fixed = nineFour_high_fixed.tolist()
                nineFour_low_fixed = nineFour_low_fixed.tolist()
                nineFour_close_fixed = nineFour_close_fixed.tolist()
                nineFour_vol_fixed = nineFour_vol_fixed.tolist()
            #print(len(nineFour_open_fixed))

            cur_open = nineFour_open_fixed[0]
            #print(cur_open)

            #writeToFile
            outF = open("SPY" + str(year) + "_all.txt", "a")
            now = datetime(2010, 1, 1, 9, 30, 0, 700401)
            hundredDaysLater = now + timedelta(minutes = 389)
            num = 0
            for dt in rrule.rrule(rrule.MINUTELY, dtstart=now, until=hundredDaysLater):
                #print(nineFour_open_fixed[num])
                outF.write(curDay)
                outF.write(';')
                outF.write(str(dt)[11:])
                outF.write(';')
                outF.write(str(nineFour_open_fixed[num]))
                outF.write(';')
                outF.write(str(nineFour_open_fixed[num] / cur_open - 1))
                outF.write(';')
                outF.write(str(nineFour_high_fixed[num]))
                outF.write(';')
                outF.write(str(nineFour_low_fixed[num]))
                outF.write(';')
                outF.write(str(nineFour_close_fixed[num]))
                outF.write(';')
                outF.write(str(nineFour_vol_fixed[num]))
                outF.write("\n")
                num = num + 1

            a = a + 1
            #plt.figure()
            #plt.plot(nineFour_open_fixed)
            #plt.title(curDay)
    #print(a)
    outF.close()
    read_file = pd.read_csv (r'/home/clarklu/Desktop/SPY' + str(year) + '_all.txt')
    read_file.to_csv (r'/home/clarklu/Desktop/SPY' + str(year) + '_all_csv.csv', index=None)
    
data_modify(2011)

Plot normalized data by minutes

In [ ]:
import pandas as pd
from datetime import date, timedelta
import matplotlib.pyplot as plt

data = pd.read_csv('/home/clarklu/Desktop/SPY_Data/txt/SPY.txt', sep=";", header=None)
data.columns = ['DATE', 'TIME', 'OPEN', 'OPEN_normalized','HIGH', 'LOW', 'CLOSE', 'VOL']
allTime = data['TIME'].tolist()
allNormalized = data['OPEN_normalized'].tolist()

#print(int(str(allTime[2])[0:2]) * 60 + int(str(allTime[2])[3:5]) - 571)#
#print('e' in allNormalized[2])

xx = [0] * 390
c = [0] * 390
#print(x)

for i in range(len(allTime)):
    if str(allTime[i])[0:2].isnumeric():
        index = int(str(allTime[i])[0:2]) * 60 + int(str(allTime[i])[3:5]) - 571
        if not '[' in allNormalized[i]:
            toAdd = float(allNormalized[i])
            #print(toAdd)
            xx[index % 390] = xx[index % 390] + toAdd
            c[index % 390] = c[index % 390] + 1
#print(x)
for j in range(len(xx)):
    xx[j] = xx[j] / c[j]
plt.plot(xx)
plt.xlabel('minute')
plt.ylabel('(Normalized)Open Ave')
plt.title('SPY 2011, 2013 - 2019')

### Part 2

In [ ]:
#doing spectrum analysis on ibm_resample.csv
# Load the Pandas libraries with alias 'pd' 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
# Preview the first 5 lines of the loaded data 
import os
os. getcwd()
import pandas as pd 
#ibm = pd.read_csv("ibm2_noNA.csv") 
#ibm.head()

import datetime
import numpy as np
import scipy as sp
import scipy.fftpack
import pandas as pd
import matplotlib.pyplot as plt

#ibm_avg = ibm.groupby('minutes').mean()
#ibm_avg.head()
import matplotlib.pyplot as plt
#ibm_avg.plot(y='val')
#plt.show()

#Spectrum analysis
#About How to Do Spectrum Analysis: https://ipython-books.github.io/101-analyzing-the-frequency-components-of-a-signal-with-a-fast-fourier-transform/
#load data
#import pandas
#ibm_avg = pandas.read_csv('ibm_avg.csv')
#print(ibm_avg)

import numpy as np 
import matplotlib.pyplot as plt 
from numpy.fft import fft, fftfreq, ifft

##setup for domain
##number of points
n = 390
x = np.linspace(570,960,390)
#print(x)
y = xx

#Gaussian Filter
#from scipy.ndimage import gaussian_filter1d
#z = gaussian_filter1d(y, 1)


##preparatory steps
#create all the necessary frequencies
freqs = fftfreq(n)

#mask array to be used for power spectra
#ignoring half the values, as they are complex conjucates of the other 
mask = freqs > 0

##FFT and power spectra calculations
#fft values
fft_vals = fft(y)
#print(fft_vals)
#fft_vals = gaussian_filter1d(fft_vals, 1)
#fft_vals_o = fft(y)

#true theoretical fft
fft_theo = 2.0*np.abs(fft_vals/n)
#fft_theo = gaussian_filter1d(fft_theo, 1)
#fft_theo_o = 2.0*np.abs(fft_vals_o/n)

plt.figure(1)
plt.title('Original Signal')
plt.plot(x,y, color = 'xkcd:salmon', label = 'original')
plt.legend()


plt.figure(2)
plt.plot(freqs, fft_vals, label = "raw fft values")
plt.title("Raw FFT Values - need more processing")

plt.figure(3)
plt.plot(freqs[mask] , fft_theo[mask], label = "true fft values")
plt.title("True FFT Values")

#plt.figure(5)
#plt.plot(freqs[mask] , fft_theo_o[mask], label = "true fft values")
#plt.title("Filtered FFT Values")


#plt.show(fft_theo[mask])

#print(fft_theo[mask])

##usage: python3 ibm_spectrum.py

In [ ]:
plt.plot(xx)

import math

from scipy.fft import fft, ifft
y = fft(xx)
a = 15
filter_1 = [1] * a
filter_0 = [0] * (390 - a)
filter_ = filter_1 + filter_0
yy = y
for i in range(390):
    yy[i] = y[i] * filter_[i]
    
xxx = ifft(yy)
for i in range(390):
    xxx[i] = math.sqrt(xxx[i] * xxx[i] * 1)
    
plt.figure(2)
plt.plot(xxx)
plt.title('SPY Filtered')

In [ ]:
#'value' is a numpy.array which you want to find its local maximum

import pandas as pd
a = [0]
for i in range(1, len(value) - 1):
    if value[i] >= value[i - 1] and value[i] >= value[i + 1]:
        a.append(i)
plt.plot(freqs[0:100] , value, label = "true fft values")
for i in range(5):
    plt.plot(a[i]/400, value[a[i]], 'o',color = "red")
data = {'Frequency':[a[0]/400, a[1]/400, a[2]/400, a[3]/400, a[4]/400],
        'Value':[value[a[0]],value[a[1]],value[a[2]],value[a[3]],value[a[4]]]
        }
dt = pd.DataFrame(data)
print(dt)

### Part 3

In [ ]:
import datetime
import numpy as np
import scipy as sp
import scipy.fftpack
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt 
from numpy.fft import fft, fftfreq, ifft
import os
os. getcwd()
import pandas as pd 


ibm = pd.read_csv("ibm2_noNA.csv") 

d=ibm['date']
dates= np.unique(d)
test1 = []

#outF = open('IBM.txt', 'w')

dates = dates[:4000]

#result = []

ddd = []

x_1 = []
yy_1 = []
x_2 = []
yy_2 = []
x_3 = []
yy_3 = []
x_4 = []
yy_4 = []
x_0 = []
yy_0 = []

for item in dates:
    ibm2 = ibm[['date','minutes','val']][ibm['date'] == item]
    ibm2 = ibm2.sort_values(by ='minutes')
    ibm2 = ibm2['val']
    y = ibm2.to_numpy()

    val_fft = sp.fftpack.fft(y)
    val_psd = np.abs(val_fft) ** 2
    fftfreq = sp.fftpack.fftfreq(len(val_psd), 1.)
    mask_freq = fftfreq[:196]
    mask_val = val_psd[:196]
    
    #plt.figure()
    #plt.plot(mask_freq, mask_val)
    
    a = [0]
    for i in range(1, len(mask_val) - 1):
      if mask_val[i] >= mask_val[i - 1] and mask_val[i] >= mask_val[i + 1]:
          a.append(i)

    #for i in range(5):
    #  plt.plot(a[i]/400, mask_val[a[i]], 'o',color = "red")


    #outF = open('IBM.txt', 'a')
    x_0.append(str(a[0]/400))
    yy_0.append(str(mask_val[a[0]]))
    x_1.append(str(a[1]/400))
    yy_1.append(str(mask_val[a[1]]))
    x_2.append(str(a[2]/400))
    yy_2.append(str(mask_val[a[2]]))
    x_3.append(str(a[3]/400))
    yy_3.append(str(mask_val[a[3]]))
    x_4.append(str(a[4]/400))
    yy_4.append(str(mask_val[a[4]]))
    #outF.write(str(item) + ";" + str(a[0]/400) + ";" + str(mask_val[a[0]]) + ";" + str(a[1]/400) + ";" + str(mask_val[a[1]]) + ";" + str(a[2]/400) + ";" + str(mask_val[a[2]]) + ";" + str(a[3]/400) + ";" + str(mask_val[a[3]]) + ";" + str(a[4]/400) + ";" + str(mask_val[a[4]]))
    #outF.write('\n')
    #a = []
    #a.append(float(a[0]/400))
    #b.append(float(mask_val[a[0]]))
#ddd = list(set(ddd))
#ddd.sort()
#print(ddd)
#print(len(ddd))
#outF.close()
import matplotlib.pyplot as plt
print(len(x_0))
print(len(yy_0))
plt.plot(x_0, yy_0, 'ro')
plt.figure()
plt.plot(x_1, yy_1, 'ro')
plt.figure()
plt.plot(x_2, yy_2, 'ro')
plt.figure()
plt.plot(x_3, yy_3, 'ro')
plt.figure()
plt.plot(x_4, yy_4, 'ro')
plt.figure()

In [ ]:
sort_yy0 = []
for i in yy_0:
    sort_yy0.append(i)
sort_yy0.sort()
sort_yy0 = list(set(sort_yy0))
sort_yy0.sort()
print(sort_yy0)

sort_yy1 = []
for i in yy_1:
    sort_yy1.append(i)
sort_yy1.sort()
sort_yy1 = list(set(sort_yy1))
sort_yy1.sort()
print(sort_yy1)

sort_yy2 = []
for i in yy_2:
    sort_yy2.append(i)
sort_yy2.sort()
sort_yy2 = list(set(sort_yy2))
sort_yy2.sort()
print(sort_yy2)

sort_yy3 = []
for i in yy_3:
    sort_yy3.append(i)
sort_yy3.sort()
sort_yy3 = list(set(sort_yy3))
sort_yy3.sort()
print(sort_yy3)

sort_yy4 = []
for i in yy_4:
    sort_yy4.append(i)
sort_yy4.sort()
sort_yy4 = list(set(sort_yy4))
sort_yy4.sort()
print(sort_yy4)

In [ ]:
data_input = []
for i in range(len(yy_0)):
    cur = []
    cur.append(x_1[i])
    cur.append(x_2[i])
    cur.append(x_3[i])
    cur.append(x_4[i])
    cur.append(yy_0[i])
    cur.append(yy_1[i])
    cur.append(yy_2[i])
    cur.append(yy_3[i])
    cur.append(yy_4[i])
    data_input.append(cur)

#from mpl_toolkits.mplot3d import Axes3D
#import matplotlib.pyplot as plt

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')

#x_axis = []
#y_axis = []
#z_axis = []
#for i in range(len(yy_0)):
#  x_axis.append(float(data_input[i][1]))
#  y_axis.append(float(data_input[i][2]))
#  z_axis.append(float(data_input[i][0]))
#ax.scatter(x_axis, y_axis, z_axis, c='r', marker='o')
#plt.show()


print(XX.shape)

from sklearn.datasets import load_digits
from sklearn.manifold import LocallyLinearEmbedding

embedding = LocallyLinearEmbedding(n_components=3)
X_transformed = embedding.fit_transform(XX[:4000])
print(X_transformed.shape)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x_axis = []
y_axis = []
z_axis = []
for i in range(len(yy_0)):
    x_axis.append(float(X_transformed[i][1]))
    y_axis.append(float(X_transformed[i][2]))
    z_axis.append(float(X_transformed[i][0]))
ax.scatter(x_axis, y_axis, z_axis, c='r', marker='o')
plt.show()

In [ ]:
XXXX = []
for i in range(len(yy_0)):
    cur = []
    cur.append(x_axis[i])
    cur.append(y_axis[i])
    cur.append(z_axis[i])
    XXXX.append(cur)


from sklearn.cluster import AgglomerativeClustering
import numpy as np
clustering = AgglomerativeClustering(5).fit(XXXX)
labels = clustering.labels_

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x_axis = 0
y_axis = 0
z_axis = 0

one = 0
two = 0
three = 0
four = 0
five = 0
for i in range(len(yy_0)):
    x_axis = (float(X_transformed[i][1]))
    y_axis = (float(X_transformed[i][2]))
    z_axis = (float(X_transformed[i][0]))
    if labels[i] == 0:
        ax.scatter(x_axis, y_axis, z_axis, c='r', marker='o')

plt.show()
print(one)
print(two)
print(three)
print(four)
print(five)

In [ ]:
green = 0
red = 0
blue = 0
for i in labels:
    if i == 0:
        red = red + 1
    elif i == 1:
        green = green + 1
    else:
        blue = blue + 1
print(red)
print(green)
print(blue)